# ConvBERT

## Importing required libaries

In [ ]:
!pip install transformers accelerate

In [ ]:
!pip install transformers
!pip install --upgrade transformers

In [ ]:
!pip install datasets

In [ ]:
!pip install pyarrow 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import torch
from nltk.corpus import stopwords
import torch.nn as nn
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from torch.optim import Adam
from torch.utils.data import TensorDataset, RandomSampler, DataLoader, SequentialSampler
from transformers import ConvBertTokenizer, ConvBertForSequenceClassification
from transformers import ConvBertTokenizer, ConvBertForMaskedLM
from transformers import ConvBertTokenizer, ConvBertModel, AdamW
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import typing
from typing import Dict
import pyarrow as pa
from datasets import Dataset
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

## Dataframe (clean_news.csv) processing-steps for training ConvBert Model

In [ ]:
data_path = 'clean_news.csv'
df = pd.read_csv(data_path, error_bad_lines=False)

<ipython-input-6-d4084b98eba0>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(data_path, error_bad_lines=False)


In [ ]:
df.head()

,Body,Label
0,karadeniz tiyatro festivali zamanı devlet tiya...,gerçek
1,sözde torbacının haklı gururu dün gece suların...,yalan
2,buseyi aramak bin liralık cihazını alıp geldi ...,gerçek
3,west ham southampton maç özet premier ligin ha...,gerçek
4,ak partili yavuz duyurdu yskya ek dilekçe vere...,gerçek


In [ ]:
df.shape

(4455, 2)

In [ ]:
df1 = pd.DataFrame(df)
print(df1.count())

Body     4455
Label    4455
dtype: int64


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('dbmdz/convbert-base-turkish-mc4-cased')

In [ ]:
text = "This is an example sentence for tokenization."
tokens = tokenizer.tokenize(text)

In [ ]:
print("Number of tokens:", len(tokens))

Number of tokens: 14


In [ ]:
print(tokens)

['Th', '##is', 'is', 'an', 'ex', '##amp', '##le', 'sent', '##ence', 'for', 'tok', '##eniz', '##ation', '.']


In [ ]:
print("Vocabulary:", tokenizer.get_vocab())

Vocabulary: {'Kent': 7997, 'alacağım': 16898, 'öksür': 18739, 'denedim': 22048, '##İLERİ': 24601, 'halktan': 28612, 'ço': 24704, '##Г': 1852, 'geçici': 7283, 'ortada': 8859, 'öğrencinin': 11955, 'yürütülen': 10025, '##ff': 10667, 'birliktelik': 18168, '##sam': 2885, '##avüz': 10146, 'mask': 12457, '##ınar': 6074, '##trik': 29103, 'artışla': 15450, 'nakli': 19870, 'buldu': 7745, 'hatırlattı': 18231, '##ileri': 2505, 'halkla': 23333, 'miktarının': 21715, '##rad': 9412, 'YEN': 28098, 'yapmanız': 8852, '##’nı': 12523, '##meyin': 6896, '##linin': 13595, 'teslimat': 15928, 'Dört': 10734, 'yıldan': 9770, 'açıklaması': 9387, 'etmekle': 17737, '##abilmeleri': 28068, 'ım': 26987, 'demeden': 23325, 'yapman': 21010, 'Kapat': 25000, 'patron': 11736, 'konumlandır': 24453, 'ramazan': 21329, 'Nor': 10693, 'sensin': 17537, '##ımını': 26606, 'adlandır': 7649, '##omotiv': 10198, 'siteden': 26571, '##larýn': 23090, 'Ran': 15378, 'işlenmiş': 21183, 'dram': 21821, 'potansiy': 7211, 'talip': 21619, 'oy': 229

In [ ]:
df.head()

,Body,Label
0,karadeniz tiyatro festivali zamanı devlet tiya...,gerçek
1,sözde torbacının haklı gururu dün gece suların...,yalan
2,buseyi aramak bin liralık cihazını alıp geldi ...,gerçek
3,west ham southampton maç özet premier ligin ha...,gerçek
4,ak partili yavuz duyurdu yskya ek dilekçe vere...,gerçek


In [ ]:
df.shape

(4455, 2)

In [ ]:
def process_data(row):
        # Clean the text
        text = row['Body']
        text = str(text)
        text = ' '.join(text.split())
        # Get tokens
        encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)
        # Convert string to integers
        label = 0
        if row['Label'] == 'gerçek':
            label += 1

        encodings['label'] = label
        encodings['text'] = text

        return encodings

In [ ]:
print(process_data({
        'Body': 'this is a body text of news.',
        'Label': 'gerçek'
    }))

{'input_ids': [2, 19792, 2605, 69, 11833, 7542, 29238, 3833, 21938, 1022, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
processed_data = []
for i in range(len(df[:4455])):
 processed_data.append(process_data(df.iloc[i]))

In [ ]:
new_df = pd.DataFrame(processed_data)
  
train_df, valid_df = train_test_split(
        new_df,
        test_size=0.2,
        random_state=2022
    )

In [ ]:
new_df.head()

,attention_mask,input_ids,label,text,token_type_ids
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 5829, 4769, 7638, 26868, 6658, 3288, 7638,...",1,karadeniz tiyatro festivali zamanı devlet tiya...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 13556, 17932, 6397, 7706, 7660, 1030, 2497...",0,sözde torbacının haklı gururu dün gece suların...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 2048, 6041, 1024, 14957, 2551, 12641, 1868...",1,buseyi aramak bin liralık cihazını alıp geldi ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 19504, 2033, 4337, 11390, 15577, 3299, 398...",1,west ham southampton maç özet premier ligin ha...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[2, 2332, 27771, 4304, 2086, 11055, 93, 25976,...",1,ak partili yavuz duyurdu yskya ek dilekçe vere...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
train_df.shape

(3564, 5)

In [ ]:
valid_df.shape

(891, 5)

In [ ]:
train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [ ]:
train_hg.shape

(3564, 6)

## Training Stage for ConvBert

In [ ]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
        'dbmdz/convbert-base-turkish-mc4-cased',
        num_labels=2
    )

Some weights of ConvBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/convbert-base-turkish-mc4-cased and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(output_dir="./result", evaluation_strategy="epoch", num_train_epochs= 5.0)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_hg,
        eval_dataset=valid_hg,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a ConvBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.043365,0.991021,0.987578,0.995825,0.991684
2,0.142500,0.155861,0.970819,0.995624,0.949896,0.972222
3,0.016300,0.058106,0.989899,0.985537,0.995825,0.990654
4,0.007100,0.063411,0.992144,0.987603,0.997912,0.992731
5,0.001800,0.061160,0.992144,0.987603,0.997912,0.992731


<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>
<class 'transformers.trainer_utils.EvalPrediction'>


TrainOutput(global_step=2230, training_loss=0.037599920495642945, metrics={'train_runtime': 548.0992, 'train_samples_per_second': 32.512, 'train_steps_per_second': 4.069, 'total_flos': 1128213024491520.0, 'train_loss': 0.037599920495642945, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.061160143464803696,
 'eval_accuracy': 0.9921436588103255,
 'eval_precision': 0.987603305785124,
 'eval_recall': 0.9979123173277662,
 'eval_f1': 0.9927310488058153,
 'eval_runtime': 7.1301,
 'eval_samples_per_second': 124.964,
 'eval_steps_per_second': 15.708,
 'epoch': 5.0}

# Creating GUI with Flask

In [ ]:
model.save_pretrained('./saved_model/')

In [ ]:
tokenizer.save_pretrained('./saved_model/')

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [ ]:
#import shutil

#shutil.make_archive("/content/saved_model/Convbertsavemodel/", 'zip', "/content/saved_model/")

In [ ]:
def edizvaluate(text):
    news_text=text
    encoded_text = tokenizer(news_text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

    # Move the model to cuda:0
    model.to('cuda:0')

    # Move the encoded_text tensors to cuda:0
    encoded_text = {k: v.to('cuda:0') for k, v in encoded_text.items()}

    # Perform inference on cuda:0
    output = model(**encoded_text)

    # Move the output logits tensor to CPU for further processing
    logits = output.logits.to('cpu')

    # Find the predicted label on CPU
    predicted_label = torch.argmax(logits).item()

    # Determine the predicted class
    predicted_class = "FAKE ❌" if predicted_label == 0 else "TRUE ✅"

    # Print the predicted class
    return predicted_class


In [ ]:
news_text= "example input text"

print("Result:", edizvaluate(news_text))

Result: TRUE ✅


In [ ]:
!pip install flask pyngrok

import socket
import os
from flask import Flask, render_template, request
from IPython.display import display, HTML
from pyngrok import ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 44.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=e3109270d09c5028d1252fcd02a2d7d37dc68f441dcea23a860a57e68c39efa2
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [ ]:
# Set your ngrok auth token
ngrok.set_auth_token("2Q4gRbz5J7KRlOCdehkXnuxG4hC_7GBJkYjnc5JnmbJkCPrDp")

app = Flask(__name__)

@app.route('/', methods=['GET'])
def home():
    return render_template('index.html')

@app.route('/evaluate', methods=['POST'])
def evaluate():
    news_text = request.form['news_text']
    result = edizvaluate(news_text)
    return render_template('result.html', result=result)

!mkdir templates

index_html = '''
<html>
<head>
<link rel="icon" type="image/x-icon" href="https://www.bilgi.edu.tr/static/images/icons/favicon-192.png">
    <title>Turkish Fake News Detector</title>
    <style>
        body {
            background-color: #f2f2f2;
            font-family: "Times New Roman", Times, serif;
            margin: 0;
            padding: 0;
        }
        .container {
            max-width: 500px;
            margin: 50px auto;
            background-color: #fff;
            padding: 20px;
            border-radius: 5px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }
        h1 {
            text-align: center;
        }
        textarea {
            width: 100%;
            height: 150px;
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 4px;
            resize: vertical;
        }
        input[type="submit"] {
            background-color: #1c589c;
            color: white;
            padding: 10px 20px;
            border: none;
            border-radius: 4px;
            cursor: pointer;
            font-size: 16px;
        }
    </style>
</head>
<body>
    <div class="container">

        <div class="image-container">
            <img src="https://tbl.bilgi.edu.tr/brand-resources/logotypes/bilgi/colored/bilgi-logotype-en.png" alt="Bilgi University Logo" style="width: 60%;">
        </div>
                <div class="header">
            <h1>Enter the News</h1>
        </div>
        <form action="/evaluate" method="post">
            <textarea name="news_text" rows="10" cols="50" required></textarea><br><br>
            <input type="submit" value="Evaluate">
        </form>
    </div>
</body>
</html>
'''

result_html = '''
<html>
<head>
<link rel="icon" type="image/x-icon" href="https://www.bilgi.edu.tr/static/images/icons/favicon-192.png">
    <title>Turkish Fake News Detector</title>
    <style>
        body {
            background-color: #f2f2f2;
            font-family: "Times New Roman", Times, serif;
            margin: 0;
            padding: 0;
        }
        .container {
            max-width: 500px;
            margin: 50px auto;
            background-color: #fff;
            padding: 20px;
            border-radius: 5px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
        }
        input[type="submit"] {
            background-color: #1c589c;
            color: white;
            padding: 10px 20px;
            border: none;
            border-radius: 4px;
            cursor: pointer;
            font-size: 16px;
        }
        h1 {
            text-align: center;
        }
        p {
            margin-top: 20px;
        }
        .fake {
            color: red;
            font-size: 20px;
            text-align: center;
        }
        .true {
            color: green;
            font-size: 20px;
            text-align: center;
        }
    </style>
</head>
<body>
    <div class="container">
<div class="image-container">
            <img src="https://tbl.bilgi.edu.tr/brand-resources/logotypes/bilgi/colored/bilgi-logotype-en.png" alt="Bilgi University Logo" style="width: 60%;">
        </div>
                <div class="header">
            <h1>The news is</h1>
        </div>
        <p class="{{ result }}">{{ result }}</p>
        <form action="/" method="GET">
            <br>
            <input type="submit" value="Back">
        </form>
    </div>
</body>

</html>
'''

with open('templates/index.html', 'w') as f:
    f.write(index_html)


with open('templates/result.html', 'w') as f:
    f.write(result_html)

@app.route('/show')
def show():
    display(HTML('<a href="/evaluate">Click here to go to the input page</a>'))

if __name__ == '__main__':
    app_port = 8000
    public_url = ngrok.connect(app_port).public_url
    print(f"Web Server URL: {public_url}")
    app.run(port=app_port, host='localhost')


Web Server URL: https://f930-34-143-243-127.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:8000
INFO:werkzeug:Press CTRL+C to quit
